In [4]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_kraken_orderbooks_from_file
from hydradx.model.processing import get_omnipool_data_from_file

from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket
from pprint import pprint

import json


In [5]:
with open('config/arbconfig.txt', 'r') as json_file:
    arb_list = json.load(json_file)
    
for d in arb_list:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_asset_list = []
buffer = {}

get_live_markets = True

if get_live_markets:

    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[arb_cfg['tkn_ids']] = arb_cfg['buffer']
        if tkn_pair not in ob_objs:
            order_book_url = 'https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1]
            ob_objs[tkn_pair] = get_kraken_orderbook(tkn_pair, order_book_url, archive=True)
            for tkn in tkn_pair:
                if tkn not in order_book_asset_list:
                    order_book_asset_list.append(tkn)
                    
else:
    ob_objs = get_kraken_orderbooks_from_file("data/input/")
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[arb_cfg['tkn_ids']] = arb_cfg['buffer']
        for tkn in tkn_pair:
            if tkn not in order_book_asset_list:
                order_book_asset_list.append(tkn)

In [7]:


RPC="wss://hydradx-rpc.dwellir.com"
N = 24

if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, N, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
buffer2 = {(asset_map[k[0]], asset_map[k[1]]): v for k, v in buffer.items()}

pprint(tokens)

{'ASTR': {'LRNA': 9426.083139248882, 'liquidity': 4487123.1018498875},
 'BNC': {'LRNA': 6603.946708737461, 'liquidity': 729547.862157833},
 'CFG': {'LRNA': 9231.929536764723, 'liquidity': 600988.2603155252},
 'DAI': {'LRNA': 12896.518469037715, 'liquidity': 338250.81980178953},
 'DAI001': {'LRNA': 31189.317908758978, 'liquidity': 816351.8852662736},
 'DOT': {'LRNA': 73125.87311942711, 'liquidity': 413407.9998085864},
 'GLMR': {'LRNA': 5351.198130481999, 'liquidity': 664616.1758693877},
 'HDX': {'LRNA': 20197.434429212783, 'liquidity': 113691495.84240358},
 'INTR': {'LRNA': 8316.189716962293, 'liquidity': 12845383.67597828},
 'USDT': {'LRNA': 84916.93279494648, 'liquidity': 2233023.664212},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41584.505065373174, 'liquidity': 31.25332448},
 'WETH': {'LRNA': 21426.620304704305, 'liquidity': 309.21785254735175},
 'WETH001': {'LRNA': 70997.18376287296, 'liquidity': 1027.4506953779776},
 'ZTG': {'LRNA': 4579.8402

In [8]:
cex_fee = 0.0016
# buffer = 0.0002

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0005
# asset_fee = 0.0025


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()
print(order_book_asset_list)


['HDX', 'USD', 'DOT', 'XETH', 'ZUSD', 'ETH', 'XXBT', 'XBT', 'ASTR', 'CFG', 'BNC', 'GLMR']


In [9]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('DOT', 'iBTC'): ('DOT', 'XBT'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUSD'),
 ('WETH', 'DAI'): ('XETH', 'ZUSD'),
 ('WETH', 'DAI001'): ('XETH', 'ZUSD'),
 ('WETH', 'USDT'): ('XETH', 'ZUS

In [10]:
all_swaps = get_arb_swaps(op_state, cex, order_book_map, buffer=buffer2)
print(all_swaps)

[{'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'WETH', 'price': 1811.61, 'amount': 0.026001930236816406, 'min_buy': 47.20918503665924}, 'cex': {'trade': 'buy', 'buy_asset': 'XETH', 'sell_asset': 'ZUSD', 'price': 1812.5158049999998, 'amount': 0.026001930236816406}}, {'dex': {'trade': 'buy', 'buy_asset': 'WBTC001', 'sell_asset': 'DAI001', 'price': 35071.9, 'amount': 0.009462417602539065, 'max_sell': 330.93249437999725}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XXBT', 'price': 35054.364050000004, 'amount': 0.009462417602539065}}, {'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'iBTC', 'price': 35072.0, 'amount': 0.018576366424560547, 'min_buy': 700.7452177078725}, 'cex': {'trade': 'buy', 'buy_asset': 'XXBT', 'sell_asset': 'ZUSD', 'price': 35089.536, 'amount': 0.018576366424560547}}, {'dex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'iBTC', 'price': 0.00013262, 'amount': 38.0859375, 'max_sell': 0.023624544024467467}, 'cex': {'trade':

In [11]:
state = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, cex, agent, all_swaps)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit = calculate_profit(init_agent, agent, asset_map)

print(profit)
            

{'HDX': 0, 'USD': 2.4484899044036865, 'BTC': 2.0265579223632812e-05, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0.0, 'INTR': 0}
